# Yaw angle optimisation
Calculating the optimum yaw angle for turbines across a range of wakes and wind speeds

## To do
- Increase fidelity
    - Assess difference in speed of calculation and optimal values
- Loop over wind direction
    - Parallelise using PQDM
- Assess results
    - Sensitivity to wind direction/speed
    - Variation in sensitivity between turbines
    - Increase number of turbines

## Setup

In [1]:
# import libraries
import logging

import numpy as np
import xarray as xr

import utils

## Baseline values

In [2]:
# run baseline simulation
sim_res_base = utils.run_sim()

In [3]:
# ensure probabilities (wind direction and speed) total 1
Sector_frequency = sim_res_base.Sector_frequency
if not np.isclose(Sector_frequency.sum(), 1):
    logging.warning(
        f"Sector frequency renormalised as total probability was {Sector_frequency.sum().values}"
    )
    Sector_frequency = Sector_frequency / Sector_frequency.sum()
P = sim_res_base.P
if not np.isclose(P.sum(), 1):
    logging.warning(f"P renormalised as total probability was {P.sum().values}")
    P = P / P.sum()

In [4]:
# calculate baseline metrics
aep_base, lcoe_base, cap_fac_base = utils.calc_metrics(
    sim_res=sim_res_base,
    sim_res_base=sim_res_base,
    Sector_frequency=Sector_frequency,
    P=P,
    show=True,
)
(
    lcoe_direction_base,
    cap_fac_direction_base,
    lcoe_overall_base,
    cap_fac_overall_base,
) = utils.aggregate_metrics(
    aep=aep_base,
    lcoe=lcoe_base,
    cap_fac=cap_fac_base,
    Sector_frequency=Sector_frequency,
)

AEP [GWh]: 79.174
LCoE [USD/MWh]: 44.477
Capacity factor [%]: 50.177


## Optimise across for a single wind direction

In [5]:
# define constants
wd = 270
yaw_shape = (len(sim_res_base.wt), 1, len(sim_res_base.ws))

In [6]:
# initialise optimal yaw dataset
yaw_opt = xr.Dataset(
    coords={
        "wt": list(sim_res_base.wt.values),
        "wd": [wd],
        "ws": list(sim_res_base.ws.values),
    },
)

In [7]:
# run optimisation and save output
yaw_opt_power, yaw_opt_lcoe = utils.optimise_direction(
    wd=wd,
    sim_res_base=sim_res_base,
    Sector_frequency=Sector_frequency,
    P=P,
    lcoe_direction_base=lcoe_direction_base.sel(wd=wd),
)
yaw_opt["power"] = (["wt", "wd", "ws"], yaw_opt_power)
yaw_opt["lcoe"] = (["wt", "wd", "ws"], yaw_opt_lcoe)

In [8]:
# rerun simulation for optimum
sim_res_opt = utils.run_sim(yaw=yaw_opt.lcoe.values.reshape(yaw_shape), wd=wd)
aep_opt, lcoe_opt, cap_fac_opt = utils.calc_metrics(
    sim_res=sim_res_opt,
    sim_res_base=sim_res_base,
    Sector_frequency=Sector_frequency,
    P=P,
)
_, _, lcoe_overall_opt, cap_fac_overall_opt = utils.aggregate_metrics(
    aep=aep_opt, lcoe=lcoe_opt, cap_fac=cap_fac_opt, Sector_frequency=Sector_frequency
)

In [9]:
# display comaprison of optimum to baseline
print(f"Baseline LCoE [USD/MWh]      : {lcoe_direction_base.sel(wd=wd).values:.3f}")
print(f"Optimum LCoE [USD/MWh]       : {lcoe_overall_opt.values:.3f}")
print(
    f"Baseline Capacity Factor [%] : {100*cap_fac_direction_base.sel(wd=wd).values:.3f}"
)
print(f"Optimum Capacity Factor [%]  : {100*cap_fac_overall_opt.values:.3f}")

Baseline LCoE [USD/MWh]      : 43.252
Optimum LCoE [USD/MWh]       : 42.918
Baseline Capacity Factor [%] : 51.598
Optimum Capacity Factor [%]  : 52.000
